In [149]:
import pandas as pd 
import plotly.express as px 
import json
import plotly
from plotly.offline import plot
from plotly.subplots import make_subplots

In [150]:
#import local csv with 2019 to 2021 population data (per state)
#dataset was joined and cleaned with SQL postgres and exported for visuals to be created here
df_pop = pd.read_csv("btu2019to2021export.csv")
df_pop

,st_abbreviation,2019,2020,2021,2022,2023,code,btu_2019,btu_2020,btu_2021
0,AK,733374,732964,734923,733276,733406,ABICB,-10.0,-7.0,-8.0
1,AK,733374,732964,734923,733276,733406,ARICB,11973.0,12391.0,13425.0
2,AK,733374,732964,734923,733276,733406,ARTCB,11973.0,12391.0,13425.0
3,AK,733374,732964,734923,733276,733406,ARTXB,11973.0,12391.0,13425.0
4,AK,733374,732964,734923,733276,733406,AVACB,1245.0,915.0,1037.0
...,...,...,...,...,...,...,...,...,...,...
12286,WY,576850,577664,579548,581629,584057,WYICB,0.0,0.0,0.0
12287,WY,576850,577664,579548,581629,584057,WYTCB,37045.0,48329.0,74713.0
12288,WY,576850,577664,579548,581629,584057,WYTXB,0.0,0.0,0.0
12289,WY,576850,577664,579548,581629,584057,ZWCDP,243.0,352.0,411.0


In [151]:
#RC is residential sector consumption
#filter energy use codes by RC, to get residential use (btu per state)
total_df_pop = df_pop[df_pop["code"].str.match("[A-Z]{2}RC[A-Z]")]
total_df_pop["code"].unique()

array(['CLRCB', 'DFRCB', 'ESRCB', 'GERCB', 'HLRCB', 'KSRCB', 'LORCB',
       'NGRCB', 'PARCB', 'PQRCB', 'RERCB', 'SFRCB', 'SORCB', 'TERCB',
       'TNRCB', 'WDRCB'], dtype=object)

In [152]:
#calculating use per person, per year
total_df_pop['2019pp']=total_df_pop['btu_2019']/total_df_pop['2019']
total_df_pop['2020pp']=total_df_pop['btu_2020']/total_df_pop['2020']
total_df_pop['2021pp']=total_df_pop['btu_2020']/total_df_pop['2021']

#create new dataframe with only 2019 to 2021 BTU/person data
df_covid = total_df_pop[['st_abbreviation','code','2019pp','2020pp','2021pp']]
#rename columns for simplification 
df_covid.columns=['state','code','2019','2020','2021']
df_covid

/var/folders/sh/lh8_sldj2tj11k741sl2jpnm0000gn/T/ipykernel_49812/2846017607.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/sh/lh8_sldj2tj11k741sl2jpnm0000gn/T/ipykernel_49812/2846017607.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/sh/lh8_sldj2tj11k741sl2jpnm0000gn/T/ipykernel_49812/2846017607.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,state,code,2019,2020,2021
27,AK,CLRCB,0.000000,0.000000,0.000000
35,AK,DFRCB,0.008964,0.009728,0.009702
55,AK,ESRCB,0.008971,0.009725,0.009699
71,AK,GERCB,0.000138,0.000138,0.000137
77,AK,HLRCB,0.000543,0.000595,0.000593
...,...,...,...,...,...
12234,WY,SFRCB,0.000000,0.000000,0.000000
12242,WY,SORCB,0.000106,0.000158,0.000157
12252,WY,TERCB,0.093494,0.090598,0.090303
12261,WY,TNRCB,0.056933,0.053580,0.053405


In [153]:
#use melt method to assign years per column of btu entry
#rearranging table will allow for plotting
df_covid_new= df_covid.melt(id_vars=["state", "code"], var_name="year", value_name="btu_per_person")
df_covid_new

,state,code,year,btu_per_person
0,AK,CLRCB,2019,0.000000
1,AK,DFRCB,2019,0.008964
2,AK,ESRCB,2019,0.008971
3,AK,GERCB,2019,0.000138
4,AK,HLRCB,2019,0.000543
...,...,...,...,...
2443,WY,SFRCB,2021,0.000000
2444,WY,SORCB,2021,0.000157
2445,WY,TERCB,2021,0.090303
2446,WY,TNRCB,2021,0.053405


In [154]:
#sum residential energy use by state and year
df_covid_grouped = df_covid_new.groupby(["state", "year"], as_index=False).sum()
df_covid_grouped


,state,year,code,btu_per_person
0,AK,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.200405
1,AK,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.224464
2,AK,2021,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.223866
3,AL,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.171314
4,AL,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.163031
...,...,...,...,...
148,WV,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.220460
149,WV,2021,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.221240
150,WY,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.263909
151,WY,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.252394


In [155]:
df_covid_grouped.to_csv('covid_grouped.csv')

In [156]:
#Use animation frame to create interactive visual that demonstrates stage energy usage per person (in ascending order)
figa = px.bar(
        df_covid_grouped, 
        x="state", 
        y="btu_per_person", 
        animation_frame="year", 
        range_y=(
            0, 
            df_covid_grouped["btu_per_person"].max()
        ), 
        color="state",
        title="United States residential energy consumption PER CAPITA (BTU)"
).update_xaxes(categoryorder="total ascending")

figa.show()

In [157]:
# Testing-save the interactive HTML file
html_file_path = "interactive_plot_FINAL.html"
plot(figa, filename=html_file_path, auto_open=False)

'interactive_plot_FINAL.html'

In [39]:
#code below to convert figure to json did not work, output was not dynamic
#plotly_json_2 = fig.to_json()

In [35]:
#code below to convert figure to json did not work, output was not dynamic
# plotly_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

In [47]:
# #code below to convert figure to json did not work, output was not dynamic
# print(plotly_json_2)

In [158]:
#create new dataframe with only 2019 to 2021 total BTU consumption per state
df_covid_stateuse = total_df_pop[['st_abbreviation','code','btu_2019','btu_2020','btu_2021']]
#rename columns for simplification 
df_covid_stateuse.columns=['state','code','2019','2020','2021']
df_covid_stateuse

,state,code,2019,2020,2021
27,AK,CLRCB,0.0,0.0,0.0
35,AK,DFRCB,6574.0,7130.0,7955.0
55,AK,ESRCB,6579.0,7128.0,7110.0
71,AK,GERCB,101.0,101.0,101.0
77,AK,HLRCB,398.0,436.0,419.0
...,...,...,...,...,...
12234,WY,SFRCB,0.0,0.0,0.0
12242,WY,SORCB,61.0,91.0,132.0
12252,WY,TERCB,53932.0,52335.0,52210.0
12261,WY,TNRCB,32842.0,30951.0,30903.0


In [159]:
#use melt method to assign years per column of btu entry
#rearranging table will allow for plotting
df_covid_stateuse_new= df_covid_stateuse.melt(id_vars=["state", "code"], var_name="year", value_name="btu_state")
df_covid_stateuse_new

,state,code,year,btu_state
0,AK,CLRCB,2019,0.0
1,AK,DFRCB,2019,6574.0
2,AK,ESRCB,2019,6579.0
3,AK,GERCB,2019,101.0
4,AK,HLRCB,2019,398.0
...,...,...,...,...
2443,WY,SFRCB,2021,0.0
2444,WY,SORCB,2021,132.0
2445,WY,TERCB,2021,52210.0
2446,WY,TNRCB,2021,30903.0


In [160]:
#sum residential energy use by year to visualize trend of residential energy consumption by year (total sum)
df_covid_stateuse_new.columns
df_covidyr_grouped = df_covid_stateuse_new.groupby(["year"], as_index=False).sum()
df_covidyr_grouped
df_covidyr_grouped.columns=['year','state','code','totalBTU']

In [161]:
figb = px.line(df_covidyr_grouped, x="year", y="totalBTU", title='US total residential energy consumption per year')
figb.show()

In [163]:
#sum residential energy use by state and year
df_covid_grouped = df_covid_new.groupby(["state", "year"], as_index=False).sum()
df_covid_grouped

,state,year,code,btu_per_person
0,AK,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.200405
1,AK,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.224464
2,AK,2021,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.223866
3,AL,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.171314
4,AL,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.163031
...,...,...,...,...
148,WV,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.220460
149,WV,2021,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.221240
150,WY,2019,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.263909
151,WY,2020,CLRCBDFRCBESRCBGERCBHLRCBKSRCBLORCBNGRCBPARCBP...,0.252394


In [164]:
#Use animation frame to create interactive visual that demonstrates stage energy usage per person (in ascending order)
fig2 = px.bar(
        df_covidst_grouped, 
        x="state", 
        y="btu_state", 
        animation_frame="year", 
        range_y=(
            0, 
            df_covidst_grouped["btu_state"].max()
        ), 
        color="state",
        title="United States residential energy consumption PER STATE (BTU)"
).update_xaxes(categoryorder="total ascending")
    
fig2.show()

In [165]:
# Save Fig 2 as an interactive HTML file
html_file_path = "interactive_plot_TOTAL_FINAL.html"
plot(fig2, filename=html_file_path, auto_open=False)

'interactive_plot_TOTAL_FINAL.html'

In [166]:
#Analyzing transportation end use energy consumption for US as a whole from pre covid to 2021:
#AC is residential sector consumption
#filter energy use codes by AC, to get residential use (btu per state)
total_df_pop_AC = df_pop[df_pop["code"].str.match("[A-Z]{2}AC[A-Z]")]
total_df_pop_AC["code"].unique()

array(['AVACB', 'BDACB', 'CLACB', 'DFACB', 'EMACB', 'ESACB', 'HLACB',
       'JFACB', 'LOACB', 'LUACB', 'MGACB', 'NGACB', 'OPACB', 'PAACB',
       'PQACB', 'REACB', 'RFACB', 'TEACB', 'TNACB'], dtype=object)

In [167]:
total_df_pop_AC

,st_abbreviation,2019,2020,2021,2022,2023,code,btu_2019,btu_2020,btu_2021
4,AK,733374,732964,734923,733276,733406,AVACB,1245.0,915.0,1037.0
9,AK,733374,732964,734923,733276,733406,BDACB,865.0,865.0,865.0
20,AK,733374,732964,734923,733276,733406,CLACB,0.0,0.0,0.0
31,AK,733374,732964,734923,733276,733406,DFACB,36849.0,30027.0,29651.0
45,AK,733374,732964,734923,733276,733406,EMACB,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
12210,WY,576850,577664,579548,581629,584057,PQACB,28.0,16.0,6.0
12218,WY,576850,577664,579548,581629,584057,REACB,3774.0,3451.0,3183.0
12225,WY,576850,577664,579548,581629,584057,RFACB,0.0,0.0,0.0
12245,WY,576850,577664,579548,581629,584057,TEACB,116164.0,107115.0,110044.0


In [168]:
#Create dataframe with only AC BTU per year across states
total_df_AC = total_df_pop_AC[['st_abbreviation','code','btu_2019','btu_2020','btu_2021']]
total_df_AC.columns=['state','code','2019','2020','2021']
total_df_AC

,state,code,2019,2020,2021
4,AK,AVACB,1245.0,915.0,1037.0
9,AK,BDACB,865.0,865.0,865.0
20,AK,CLACB,0.0,0.0,0.0
31,AK,DFACB,36849.0,30027.0,29651.0
45,AK,EMACB,0.0,0.0,0.0
...,...,...,...,...,...
12210,WY,PQACB,28.0,16.0,6.0
12218,WY,REACB,3774.0,3451.0,3183.0
12225,WY,RFACB,0.0,0.0,0.0
12245,WY,TEACB,116164.0,107115.0,110044.0


In [169]:
#use melt method to assign years per column of btu entry
total_df_AC_new= total_df_AC.melt(id_vars=["state", "code"], var_name="year", value_name="btu_US")
total_df_AC_new

,state,code,year,btu_US
0,AK,AVACB,2019,1245.0
1,AK,BDACB,2019,865.0
2,AK,CLACB,2019,0.0
3,AK,DFACB,2019,36849.0
4,AK,EMACB,2019,0.0
...,...,...,...,...
2902,WY,PQACB,2021,6.0
2903,WY,REACB,2021,3183.0
2904,WY,RFACB,2021,0.0
2905,WY,TEACB,2021,110044.0


In [170]:
#sum transportation energy consumption by year to compare to findings from consumer spending trends
total_df_AC_grouped = total_df_AC_new.groupby(["year"], as_index=False).sum()
total_df_AC_grouped

,year,state,code,btu_US
0,2019,AKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKALALALAL...,AVACBBDACBCLACBDFACBEMACBESACBHLACBJFACBLOACBL...,116038189.0
1,2020,AKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKALALALAL...,AVACBBDACBCLACBDFACBEMACBESACBHLACBJFACBLOACBL...,98852260.0
2,2021,AKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKALALALAL...,AVACBBDACBCLACBDFACBEMACBESACBHLACBJFACBLOACBL...,109780954.0


In [171]:
#create line plot to commpare against consumer spending trends in separate section of this project's covid analysis
fig3 = px.line(total_df_AC_grouped, x="year", y="btu_US", title='US Transportation energy consumption per year')
fig3.show()

In [91]:
# Save Fig 3 as an interactive HTML file
html_file_path = "static_UStransportation_Energy.html"
plot(fig3, filename=html_file_path, auto_open=False)

'static_UStransportation_Energy.html'

In [148]:
#export mutiple plots on one div
with open('multiplefigs.html', 'a') as f:
    f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figa.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figb.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig3.to_html(full_html=False, include_plotlyjs='cdn'))